In [1]:
import gensim

C:\Users\vgnsh\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sqlite3
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

import re
import nltk
from nltk.corpus import stopwords

In [3]:
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
final= pd.read_sql_query("""SELECT * FROM Reviews""",conn)
conn.close()
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText_text
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these sendak books and watching t...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,this fun way for children learn their months t...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,this great little book read has nice rhythm we...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,this book poetry about the months the year goe...


### preprocessing for summary

In [4]:
stop=set(stopwords.words('english'))
print(stop)

{'with', 'hadn', 'shan', 'weren', 'didn', 'herself', 've', 'himself', 't', 'been', 'those', 'will', 'wasn', 'who', 'being', 'had', 'as', 'by', 'after', 'only', 'out', 'where', 're', "hasn't", 'does', "mustn't", "wouldn't", 'y', 'than', "it's", 'each', 'a', "weren't", 'when', 'until', 'am', 'themselves', 'here', 'ain', 'they', 'if', 'him', 'll', 'has', 'you', 'both', 'we', 'can', 'all', 'from', 'doesn', 'up', "haven't", 'hasn', "won't", 'an', 'and', 'on', 'their', 'for', 'off', 'her', "that'll", "you've", 'needn', 'd', "needn't", 'that', 'what', 'be', 'the', "she's", 'any', 'couldn', 'own', 'yourself', 'not', 'is', 'them', 'once', 'yours', 'shouldn', 'ourselves', 'ma', 'myself', 'there', 'ours', 'no', 'how', 'yourselves', 'in', "aren't", 'have', 'aren', 'she', 'more', 'so', 'should', 'nor', 'then', "you'll", 'having', 'don', 'few', 'whom', 'down', 'my', 'your', 'were', "you're", 'which', 'about', 'over', 'doing', 'its', 'these', 'me', 'are', 'mustn', 'other', 'against', 'but', 'isn', 'a

In [5]:
lst=['won','nor','not','against']
for word in lst:
    stop.remove(word)
print(stop)

{'with', 'hadn', 'shan', 'weren', 'didn', 'herself', 've', 'himself', 't', 'been', 'those', 'will', 'wasn', 'who', 'being', 'had', 'as', 'by', 'after', 'only', 'out', 'where', 're', "hasn't", 'does', "mustn't", "wouldn't", 'y', 'than', "it's", 'each', 'a', "weren't", 'when', 'until', 'am', 'themselves', 'here', 'ain', 'they', 'if', 'him', 'll', 'has', 'you', 'both', 'we', 'can', 'all', 'from', 'doesn', 'up', "haven't", 'hasn', "won't", 'an', 'and', 'on', 'their', 'for', 'off', 'her', "that'll", "you've", 'needn', 'd', "needn't", 'that', 'what', 'be', 'the', "she's", 'any', 'couldn', 'own', 'yourself', 'is', 'them', 'once', 'yours', 'shouldn', 'ourselves', 'ma', 'myself', 'there', 'ours', 'no', 'how', 'yourselves', 'in', "aren't", 'have', 'aren', 'she', 'more', 'so', 'should', 'then', "you'll", 'having', 'don', 'few', 'whom', 'down', 'my', 'your', 'were', "you're", 'which', 'about', 'over', 'doing', 'its', 'these', 'me', 'are', 'mustn', 'other', 'but', 'isn', 'again', 'because', 'just',

In [6]:
sno=nltk.stem.SnowballStemmer('english')

In [7]:
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext

def cleanpunc(sentence):
    '''This function cleans all the punctuation or special characters from a given sentence'''
    cleaned = re.sub(r'[?|@|!|^|%|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [10]:
def preprocessing(series):
    i=0
    str1=" "
    final_string2=[]
    list_of_sent2=[]
    all_positive_words=[]
    all_negative_words=[]
    
    for sent in series.values:
        filtered_sent=[]
        list_of_sent=[]
        sent=cleanhtml(sent)
        sent=cleanpunc(sent)
        for cleaned_words in sent.split():
             if((cleaned_words.isalpha()) & (len(cleaned_words) > 2)):
                    if(cleaned_words.lower() not in stop):
                        s = (sno.stem(cleaned_words.lower()))
                        filtered_sent.append(s)    
                        if (final['Score'].values)[i] == 'positive':
                            all_positive_words.append(s)
                        if(final['Score'].values)[i] == 'negative':
                            all_negative_words.append(s)
        list_of_sent2.append(filtered_sent)
        str1 = " ".join(filtered_sent)
        final_string2.append(str1)
        i += 1
    return final_string2, list_of_sent2  

In [11]:
final_string2,list_of_sent2=preprocessing(final['Summary'])

In [12]:
final['CleanedSummary']=final_string2

In [13]:
conn = sqlite3.connect('final2.sqlite')
c=conn.cursor()
final.to_sql('Reviews', conn, if_exists='replace')
conn.close()

In [14]:
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText_text,CleanedSummary
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...,everi book educ
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these sendak books and watching t...,love book miss hard cover version
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,this fun way for children learn their months t...,chicken soup rice month
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,this great little book read has nice rhythm we...,good swingi rhythm read aloud
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,this book poetry about the months the year goe...,great way learn month


In [15]:
list_of_sent2[:5]

[['everi', 'book', 'educ'],
 ['love', 'book', 'miss', 'hard', 'cover', 'version'],
 ['chicken', 'soup', 'rice', 'month'],
 ['good', 'swingi', 'rhythm', 'read', 'aloud'],
 ['great', 'way', 'learn', 'month']]

### avg w2v for summary and reviews

In [16]:
with open('list_of_sent_for_input_to_w2v.pkl', 'rb') as pickle_file:
    list_of_sent = pickle.load(pickle_file)

In [17]:
def calc_avg_w2v(list_of_sent, w2v_model):
      
    sent_vectors = []
    for sent in list_of_sent:
        sent_vec = np.zeros(50)
        cnt_words = 0
        for word in sent:
            try:
                vec = w2v_model.wv[word]
                sent_vec += vec
                cnt_words += 1
            except:
                pass
        sent_vec /= cnt_words
        sent_vectors.append(sent_vec)
    return sent_vectors

#### w2v for reviews

In [19]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [20]:
sent_vectors = calc_avg_w2v(list_of_sent, w2v_model)

C:\Users\vgnsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


#### w2v for summary

In [21]:
w2v_model2=gensim.models.Word2Vec(list_of_sent2,min_count=5,size=50, workers=4)

In [22]:
sent_vectors2 = calc_avg_w2v(list_of_sent2, w2v_model2)

C:\Users\vgnsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


### preparing knn for cleaned summary and reviews

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score

C:\Users\vgnsh\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText_text,CleanedSummary
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...,everi book educ
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these sendak books and watching t...,love book miss hard cover version
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,this fun way for children learn their months t...,chicken soup rice month
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,this great little book read has nice rhythm we...,good swingi rhythm read aloud
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,this book poetry about the months the year goe...,great way learn month


In [25]:
final=final.drop('index',axis=1)

In [26]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText_text,CleanedSummary
0,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witty little book makes son laugh loud re...,everi book educ
1,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading these sendak books and watching t...,love book miss hard cover version
2,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,this fun way for children learn their months t...,chicken soup rice month
3,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,this great little book read has nice rhythm we...,good swingi rhythm read aloud
4,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,this book poetry about the months the year goe...,great way learn month


#### replacing positive with 1 and negative with 0

In [27]:
def polarity(x):
    if x=='positive':
        return 1
    return 0
    

In [28]:
final['Score']=final['Score'].map(polarity)

#### separating time and score 

In [29]:
df = final[['Time','Score']]
df.head(10)

,Time,Score
0,939340800,1
1,1194739200,1
2,1191456000,1
3,1076025600,1
4,1018396800,1
5,1344211200,1
6,1338249600,1
7,1325721600,1
8,1313884800,1
9,1303171200,1


#### dataframe for avg w2v(cleaned reviews)

In [30]:
df1=pd.DataFrame(sent_vectors)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.360383,0.805195,-0.078264,0.116988,-0.742278,0.110265,0.269673,-0.852107,0.386381,0.952745,...,-0.628075,0.347953,0.423479,0.452810,0.099181,-0.218314,0.399080,-0.206204,0.025045,0.086309
1,0.261175,0.706699,-0.828445,0.136460,-0.467704,0.309304,-0.232039,-0.270088,0.387445,0.740936,...,0.280547,0.070732,0.485283,0.820898,-0.583669,-0.771335,-0.186891,0.420213,0.087552,0.293366
2,0.404462,0.625804,-1.285407,-0.506338,-0.780310,-0.921583,-0.108740,-0.273456,0.698334,1.102643,...,-0.469223,0.494894,0.832264,0.219944,-1.161500,-0.538775,0.742973,-0.951416,0.115099,0.345993
3,0.000038,0.576355,-0.793222,-0.415365,-0.191868,0.020085,-0.377755,-0.049539,0.258079,0.542514,...,0.306974,0.000444,0.278126,0.372895,-0.734748,-0.091716,-0.788149,0.044087,0.157675,0.087588
4,0.081338,1.058320,-0.337630,-0.785302,-0.195855,-0.860340,-0.322493,-0.228184,0.068938,0.631048,...,-0.622095,0.286809,0.520046,0.709922,-0.576136,-0.243810,0.466086,-0.208573,0.033112,0.165652


#### dataframe for w2v(cleaned summary)

In [31]:
df2=pd.DataFrame(sent_vectors2)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.048758,0.270944,0.013148,-0.102761,-0.140179,0.338846,0.033525,0.217252,-0.814933,0.202458,...,0.179360,0.479057,-0.220772,0.024111,0.137097,0.251441,-0.079335,-0.049416,0.232937,0.168686
1,-0.111193,-0.341639,0.134958,-0.023534,0.353248,0.482056,0.400834,0.136476,-0.279498,-0.092229,...,0.135926,0.749689,-0.834106,0.414804,-0.414607,-0.238482,-0.078463,0.112126,0.477197,-0.481466
2,0.296324,0.911287,-1.097433,-1.005536,-0.698014,0.301079,0.229481,1.220347,-0.038344,0.204688,...,-0.575279,0.779125,-0.612699,0.133846,0.245791,1.121688,0.039687,0.538211,-0.081592,-0.765912
3,-0.208783,-0.334745,1.082776,-0.074244,0.170070,-0.309294,0.060269,0.083476,-0.849359,-0.515192,...,-0.431247,0.796204,-0.808478,0.844168,0.042495,-0.387496,0.197219,0.123357,0.294431,0.009424
4,0.453784,0.319845,0.320954,-0.133412,0.271481,0.307536,-0.165772,0.432793,-0.547968,0.307986,...,-0.309135,0.683359,-0.589170,0.132126,0.072871,0.291954,0.332793,-0.581658,0.622461,-0.195739


#### concatenating all dataframes

In [32]:
df=pd.concat([df,df1,df2],axis=1)

In [33]:
df.head()

,Time,Score,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,939340800,1,0.360383,0.805195,-0.078264,0.116988,-0.742278,0.110265,0.269673,-0.852107,...,0.179360,0.479057,-0.220772,0.024111,0.137097,0.251441,-0.079335,-0.049416,0.232937,0.168686
1,1194739200,1,0.261175,0.706699,-0.828445,0.136460,-0.467704,0.309304,-0.232039,-0.270088,...,0.135926,0.749689,-0.834106,0.414804,-0.414607,-0.238482,-0.078463,0.112126,0.477197,-0.481466
2,1191456000,1,0.404462,0.625804,-1.285407,-0.506338,-0.780310,-0.921583,-0.108740,-0.273456,...,-0.575279,0.779125,-0.612699,0.133846,0.245791,1.121688,0.039687,0.538211,-0.081592,-0.765912
3,1076025600,1,0.000038,0.576355,-0.793222,-0.415365,-0.191868,0.020085,-0.377755,-0.049539,...,-0.431247,0.796204,-0.808478,0.844168,0.042495,-0.387496,0.197219,0.123357,0.294431,0.009424
4,1018396800,1,0.081338,1.058320,-0.337630,-0.785302,-0.195855,-0.860340,-0.322493,-0.228184,...,-0.309135,0.683359,-0.589170,0.132126,0.072871,0.291954,0.332793,-0.581658,0.622461,-0.195739


#### sorting the dataframe with respect to time

In [34]:
df.sort_values('Time',inplace=True)
df.head(5)

,Time,Score,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,939340800,1,0.360383,0.805195,-0.078264,0.116988,-0.742278,0.110265,0.269673,-0.852107,...,0.179360,0.479057,-0.220772,0.024111,0.137097,0.251441,-0.079335,-0.049416,0.232937,0.168686
30,940809600,1,0.714295,0.492165,-0.571424,-0.010573,-0.727590,-0.719073,0.218302,-0.789182,...,0.096922,0.428003,-0.468993,0.384125,-0.049822,0.145312,0.123209,-0.473117,0.428591,-0.216979
424,944092800,1,1.351956,1.137352,1.114389,-0.417058,0.097867,-0.195420,-0.228267,0.458588,...,-0.249160,0.758557,-0.380744,0.246863,-0.181934,-0.382965,0.127880,0.636222,0.199713,0.203947
330,944438400,1,0.373569,0.670699,0.934661,-0.203542,-0.263391,0.188546,0.707733,0.452666,...,0.064320,0.931868,-0.222611,-0.023340,0.220713,0.258140,-0.164017,-0.330855,0.240056,0.057455
423,946857600,1,1.128998,1.503817,0.326526,-0.700867,0.096427,0.100782,-0.112834,0.344407,...,-0.314511,0.113347,-1.150788,-0.155254,0.184869,0.841742,0.325115,-0.069460,-0.542017,-0.398297


In [35]:
df['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

#### shuffling the dataframe

In [36]:
from sklearn.utils import shuffle
df = shuffle(df,random_state=123)

In [37]:
df.head()

,Time,Score,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
108163,1328832000,1,0.249139,-0.151107,-0.593884,-0.674006,0.239964,0.418372,-0.488673,0.022548,...,-0.010642,0.748598,-1.717001,0.185110,-0.270519,-0.204314,0.023914,0.400330,-0.429526,0.226474
306265,1290297600,1,0.395948,0.801223,-0.320076,-0.238848,0.069623,-0.264822,-0.349094,-0.131822,...,-0.279416,0.680942,-0.378827,0.445362,0.077234,-0.489826,-0.219724,-0.291478,0.728811,-0.290324
195553,1284249600,1,0.513640,0.621714,-0.321352,0.025803,0.518928,1.735829,-0.719569,-0.096853,...,0.381369,0.975368,-0.499815,0.368569,0.217888,0.258769,0.582724,0.124059,0.541937,-0.649226
93682,1271376000,1,0.076850,0.030091,-0.436612,-0.223092,0.242067,-0.430782,0.574311,0.628714,...,-0.260516,0.688271,-0.832284,0.505065,-0.334511,0.608125,-0.086179,0.340272,0.392449,0.308328
283000,1301961600,1,0.726777,-0.499065,-0.933332,-0.035607,0.053519,2.053453,-0.324685,0.115133,...,0.104985,1.158669,-0.457662,0.214108,0.277255,-0.776639,0.058944,0.233333,-0.116754,-0.297725


#### droping nan values

In [41]:
df=df.dropna()

In [45]:
df.shape

(358786, 102)

#### building training and testing data

In [42]:
x=df.iloc[:,2:].values
y=df.iloc[:,1].values

In [43]:
type(y)

numpy.ndarray

#### column standardizing x

In [44]:
s = StandardScaler()
x=s.fit_transform(x)

#### 70% data for training and 30% data for testing

In [46]:
x_train = x[:14000,:]
x_test = x[14000:20000,:]
y_train = y[:14000]
y_test = y[14000:20000]

In [47]:
# creating odd list of K for KNN
neighbors = list(range(1,50,2))
# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)


The optimal number of neighbors is 9.


In [48]:
y_train.shape

(14000,)

In [49]:
classifier = KNeighborsClassifier(n_neighbors = optimal_k)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')

In [50]:
y_pred = classifier.predict(x_test)
acc = accuracy_score(y_test, y_pred, normalize=True) * float(100)
acc

89.633333333333326